### Script to scrape published data from NHSD webpages and output a compiled and procesed CSV of FTE days available and FTE days lost by staff group and organisation

In [ ]:
import requests, os
from bs4 import BeautifulSoup
import re

In [ ]:
check_url = 'https://digital.nhs.uk/data-and-information/publications/statistical/nhs-sickness-absence-rates/'
file_source_url = 'https://digital.nhs.uk'

response = requests.get(check_url)

soup = BeautifulSoup(response.content, 'html.parser')
past_links = soup.find( id="past-publications").find_all(href=re.compile("publications/statistical/nhs-sickness-absence-rates/"))
latest_link = soup.find( id="latest-statistics").find_all(href=re.compile("publications/statistical/nhs-sickness-absence-rates/"))

In [ ]:
#past_links[0:10]

In [ ]:
from datetime import datetime

In [ ]:
def retrieve_sub_url(url_string, file_source_url, data_type):

    full_url = file_source_url+url_string
    #print(full_url)
    response = requests.get(full_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    csv_search = soup.find_all(href=re.compile("\.csv$"))
    xls_search = soup.find_all(href=re.compile("\.xlsx$"))
    result = [x['href'] for x in csv_search if data_type in x['href'].lower()]
    if len(result) == 0:
        result = [x['href'] for x in xls_search if data_type in x['href'].lower()]
    try:
        return(result[0])
    except:
        pass

In [ ]:
def retrieve_stats_urls(url_string, file_source_url, data_type):
    x = url_string.split('/')
    #print(x[-1])
    date_string = x[-1]
    regex_date_string = [m.group() for m in re.finditer("((january|february|march|april|may|june|july|august|september|october|november|december)-\d{4})", date_string)]
    #print(regex_date_string)

    try:
        # Retrieve find date, which takes account of URLs with a date range within
        formatted_date = datetime.strptime(regex_date_string[-1], "%B-%Y")
        #print(formatted_date)
        sub_url = retrieve_sub_url(url_string, file_source_url, data_type)

        return {
            'type': data_type,
            'file_url': sub_url,
            'the_date': formatted_date
        }
    except:
        print('Could not format date')
        pass

In [ ]:
reason_stats_urls_list = []

for link in past_links:
    print(link['href'])
    get_url = retrieve_stats_urls(link['href'], file_source_url, "reason")
    if get_url is not None:
        reason_stats_urls_list.append(get_url)

In [ ]:
len(reason_stats_urls_list)

In [ ]:
reason_stats_urls_list